In [1]:
import pandas as pd
import pyautogui
import time
import mysql.connector
from datetime import datetime, timedelta
import pyperclip
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
import time
from selenium.webdriver.common.keys import Keys
from pyworkforce.queuing import ErlangC
import numpy as np
import os
import pyperclip
from tkinter import filedialog
import customtkinter as ctk
from mysql.connector import Error
from tkcalendar import Calendar



### Configuraciones

In [2]:
publicar = 1

### Interface de elección de intervalos de consulta

In [3]:
import customtkinter as ctk

# Establecer el modo oscuro y el tema de color
ctk.set_appearance_mode("dark")  # Opciones: "dark", "light", "system"
ctk.set_default_color_theme("dark-blue")  # Puedes cambiar el tema de color según tus preferencias

# Función para guardar las configuraciones
def guardar_configuraciones():
    global inicio, fin, incidencia, interna, fecha
    inicio = inicio_var.get()
    fin = fin_var.get()
    incidencia = incidencia_var.get()
    interna = interna_var.get()
    fecha = cal.get_date()
    config_window.destroy()
    print(f"Inicio: {inicio}, Fin: {fin}, Existe incidencia: {incidencia}, Es interna: {interna}, Fecha: {fecha}")

# Función para actualizar la visibilidad del campo "Es interna:"
def actualizar_interna(*args):
    if incidencia_var.get() == "Si":
        interna_label.pack(pady=5)
        interna_dropdown.pack(pady=5)
    else:
        interna_label.pack_forget()
        interna_dropdown.pack_forget()
        interna_var.set("No")

# Crear la ventana de configuración
def abrir_configuraciones():
    global config_window, inicio_var, fin_var, incidencia_var, interna_var, interna_label, interna_dropdown, cal

    config_window = ctk.CTk()
    config_window.title("Configuraciones")

    # Configurar la ventana en pantalla centrada
    config_window.geometry("400x700")
    config_window.eval('tk::PlaceWindow . center')  # Centrando la ventana
    config_window.attributes('-topmost', True)  # Mantener la ventana en primer plano

    # Crear un marco principal con bordes
    config_frame = ctk.CTkFrame(config_window, corner_radius=10)
    config_frame.pack(padx=10, pady=10, expand=True, fill="both")

    # Crear un título
    title_label = ctk.CTkLabel(config_frame, text="Configuraciones", font=("Helvetica", 18, "bold"))
    title_label.pack(pady=10)

    # Variables para los intervalos
    inicio_var = ctk.StringVar(value="08:00")
    fin_var = ctk.StringVar(value="10:00")

    # Crear una lista de opciones de tiempo en intervalos de media hora
    time_options = [f"{h:02}:{m:02}" for h in range(8, 22) for m in (0, 30)]

    # Crear desplegables para seleccionar inicio y fin
    ctk.CTkLabel(config_frame, text="Inicio:", font=("Helvetica", 14)).pack(pady=5)
    inicio_dropdown = ctk.CTkComboBox(config_frame, values=time_options, variable=inicio_var)
    inicio_dropdown.pack(pady=5)

    ctk.CTkLabel(config_frame, text="Fin:", font=("Helvetica", 14)).pack(pady=5)
    fin_dropdown = ctk.CTkComboBox(config_frame, values=time_options, variable=fin_var)
    fin_dropdown.pack(pady=5)

    # Variables para las incidencias
    incidencia_var = ctk.StringVar(value="No")
    interna_var = ctk.StringVar(value="No")

    # Crear desplegables para seleccionar incidencia e interna
    ctk.CTkLabel(config_frame, text="Existe incidencia:", font=("Helvetica", 14)).pack(pady=5)
    incidencia_dropdown = ctk.CTkComboBox(config_frame, values=["No", "Si"], variable=incidencia_var)
    incidencia_dropdown.pack(pady=5)
    incidencia_var.trace_add("write", actualizar_interna)

    # Crear etiqueta y desplegable para "Es interna"
    interna_label = ctk.CTkLabel(config_frame, text="Es interna:", font=("Helvetica", 14))
    interna_dropdown = ctk.CTkComboBox(config_frame, values=["No", "Si"], variable=interna_var)
    
    # Inicialmente ocultar el campo "Es interna:"
    interna_label.pack_forget()
    interna_dropdown.pack_forget()

    # Añadir un calendario para seleccionar la fecha
    ctk.CTkLabel(config_frame, text="Fecha:", font=("Helvetica", 14)).pack(pady=5)
    cal = Calendar(config_frame, selectmode='day', date_pattern='yyyy-mm-dd')
    cal.pack(pady=5)

    # Crear un marco para el botón de aceptar
    button_frame = ctk.CTkFrame(config_window, corner_radius=10)
    button_frame.pack(padx=10, pady=10, fill="x")

    # Añadir botón para guardar configuraciones
    accept_button = ctk.CTkButton(button_frame, text="Aceptar", command=guardar_configuraciones)
    accept_button.pack(pady=10)

    # Ejecutar la ventana de configuración
    config_window.mainloop()

# Abrir la ventana de configuraciones
abrir_configuraciones()


Inicio: 17:00, Fin: 17:00, Existe incidencia: No, Es interna: No, Fecha: 2024-08-27


In [4]:
# Convertir la cadena a un objeto datetime
fecha_datetime = datetime.strptime(fecha, "%Y-%m-%d")

# Formatear la fecha al formato deseado
todayCMS = fecha_datetime.strftime("%d/%m/%Y")
todaySQL = fecha_datetime.strftime("%Y/%m/%d")

print(f'Fecha SQL: {todaySQL}---- Fecha CMS: {todayCMS}')

Fecha SQL: 2024/08/27---- Fecha CMS: 27/08/2024


In [5]:
# Marca el inicio del tiempo
start_time = time.time()

### Conexión a base de datos

In [6]:
# Conexión a la base de datos MySQL
def iniciarconexion(host,database,username,password):
    cnx = mysql.connector.connect(host=host, database=database, user=username, password=password)
    return cnx

#Iniciar Conexión
cnx = iniciarconexion('26.89.137.57','db_atp','gtr1','sqlPassGTR2024*')

### Crear dataframes vacios

In [7]:
#----------- CREAR DF VACIO PARA CONCATENAR REPORTES DE SPLITSKILL -----------

columnasSplitSkill = [
    'Fecha', 'Skill', 'Intervalo', 'PosicionStaff', 'OCUP', 'Tiempodispon',
    'LlamadasAtendidas', 'TMONuevo', 'TiempoStaffed', 'TiempoDisponible',
    'TiempoRing', 'TiempoACD', 'TiempoAcw', 'Tiempodereten', 
    'TiempoLlamadaSaliente', 'TiempoAux', 'AgentesDisponibles', 
    'LlamadasRecibidas'
]

# Crear un DataFrame vacío con las columnas especificadas
df_splitSkill = pd.DataFrame(columns=columnasSplitSkill)


#----------- CREAR DF VACIO PARA CONCATENAR REPORTES DE NDS -----------

columnasNDS = ['Fecha', 'Skill', 'Intervalo', 'porcObjetivo', 'objSegundos', 'NDS',
       'Llegadas', 'Abandonadas', 'Flujodesalida', 'tiempoPromedioAbandono',
       'velPromedioDeRespuesta', 'demoraMaxima', 'tiempoPromedioReten',
       'entregasInterrumpidas']

df_NDS =  pd.DataFrame(columns=columnasNDS)


In [8]:

#Función que espera a que aparezca la imagen y luego da click.
def click_on_image_when_visible(image_path):
    while True:
        location = pyautogui.locateOnScreen(image_path) 

        if location is not None:
            center = pyautogui.center(location)
            pyautogui.click(center)
            break  # Sale del bucle una vez que se encuentra la imagen

# Renombrar las columnas eliminando caracteres definidos
def remove_quotes_from_column_names(df):
    df.rename(columns=lambda x: x.replace("'", ""), inplace=True)
    df.rename(columns=lambda x: x.replace(" ", ""), inplace=True)
    df.rename(columns=lambda x: x.replace(".", ""), inplace=True)
    df.rename(columns=lambda x: x.replace("%", ""), inplace=True)


"""Verifica si el valor es una cadena no vacía y, si es así, reemplaza las comas por puntos y convierte la cadena en un valor de tipo float. 
Si el valor no es una cadena no vacía, simplemente devuelve 0.0."""
def convertir_tiempo(valor):
    if isinstance(valor, str) and valor != '':
        return float(valor.replace(',', '.'))
    else:
        return 0.0
    

def splitSkill_NDS(skill,fecha):
    pyautogui.write(skill)
    time.sleep(0.05)
    pyautogui.press("tab")
    time.sleep(0.05)
    pyautogui.write(fecha)
    
    time.sleep(0.05)
    pyautogui.press("enter")
    time.sleep(1)
    pyautogui.press("alt")
    time.sleep(0.05)
    pyautogui.press("right")
    time.sleep(0.05)
    pyautogui.press("enter")
    time.sleep(0.05)
    pyautogui.press("down")
    time.sleep(0.05)
    pyautogui.press("down")
    time.sleep(0.05)
    pyautogui.press("enter")
    time.sleep(0.05)
    pyautogui.press("enter")
    time.sleep(0.05)
    pyautogui.press("enter")
    time.sleep(0.5)

    #Reiniciar reporte
    pyautogui.press("alt")
    time.sleep(0.05)
    pyautogui.press("enter")
    time.sleep(0.05)
    pyautogui.press("enter")
    time.sleep(1)


def leer_procesar_concatenar_SplitSkill(skill,fecha, df_final):

    df = pd.read_clipboard()

    #Filtrar el df quedandome con todo menos la primer fila que es el total
    df = df.iloc[1:]

    # Insertar la columna 'Fecha' en la segunda posición
    df.insert(0, 'Fecha', fecha)

    # Insertar la columna 'Skill' en la primera posición
    df.insert(1, 'Skill', skill)

    #Eliminar caracteres especiales de los nombres de columnas
    remove_quotes_from_column_names(df)

    # Reemplaza comas por puntos en todo el DataFrame
    df = df.replace(',', '.', regex=True)

    #Convertir el tipo de datos

    dtype_mapping = {
        'PosicionStaff': int,
        'OCUP': float,
        'Tiempodispon': float,
        'LlamadasAtendidas': int,
        'TMONuevo': float,
        'TiempoStaffed': float,
        'TiempoDisponible': float,
        'TiempoRing': float,
        'TiempoACD': float,
        'TiempoAcw': float,
        'Tiempodereten': float,
        'TiempoLlamadaSaliente': float,
        'TiempoAux': float,
        'AgentesDisponibles': int,
        'LlamadasRecibidas': int,
    
        }
    

    # Convertir tipos de datos en todo el DataFrame
    df = df.astype(dtype_mapping)

    nuevos_nombres = {
            'Hora': 'Intervalo'

            }

    # Usamos el método 'rename' para cambiar los nombres de las columnas
    df = df.rename(columns=nuevos_nombres)

    # Filtrar dataframe con las columnas seleccionadas

    columns = ['Fecha', 'Skill', 'Intervalo', 'PosicionStaff', 'OCUP', 'Tiempodispon',
       'LlamadasAtendidas', 'TMONuevo', 'TiempoStaffed',
       'TiempoDisponible', 'TiempoRing', 'TiempoACD', 'TiempoAcw',
       'Tiempodereten', 'TiempoLlamadaSaliente', 'TiempoAux',
       'AgentesDisponibles', 'LlamadasRecibidas']

    df = df[columns]

    #Concatenar df con df_SplitSkill
    df_final = pd.concat([df_final, df], ignore_index=True) 

    return df_final


def leer_procesar_concatenar_NDS(skill,fecha, df_final):

    df = pd.read_clipboard()

    #Filtrar el df quedandome con todo menos la primer fila que es el total
    df = df.iloc[1:]

    # Insertar la columna 'Fecha' en la segunda posición
    df.insert(0, 'Fecha', fecha)

    # Insertar la columna 'Skill' en la primera posición
    df.insert(1, 'Skill', skill)

    #Eliminar caracteres especiales de los nombres de columnas
    remove_quotes_from_column_names(df)

    # Reemplaza comas por puntos en todo el DataFrame
    df = df.replace(',', '.', regex=True)

    nuevos_nombres = {
            'Tiempo':'Intervalo',
            'delobjetivo': 'porcObjetivo',
            'Objetivo(segundos)':'objSegundos',
            'dentrodelniveldeservicio':'NDS',
            'Llamadasaban':'Abandonadas',
            'Tiempopromdeaban':'tiempoPromedioAbandono',
            'Velpromderesp':'velPromedioDeRespuesta',
            'Demoramáxima':'demoraMaxima',
            'Tiempopromderetenc':'tiempoPromedioReten',
            'Entregasinterrumpidas':'entregasInterrumpidas'

            }

    # Usamos el método 'rename' para cambiar los nombres de las columnas
    df = df.rename(columns=nuevos_nombres)


    columnas = ['Fecha', 'Skill', 'Intervalo', 'porcObjetivo',
        'objSegundos', 'NDS', 'Llegadas', 'Abandonadas', 'Flujodesalida',
        'tiempoPromedioAbandono', 'velPromedioDeRespuesta', 'demoraMaxima',
        'tiempoPromedioReten', 'entregasInterrumpidas']

    df = df[columnas]


    df['velPromedioDeRespuesta'] = df['velPromedioDeRespuesta'].astype(float)
    df['demoraMaxima'] = df['demoraMaxima'].astype(int)
    df['tiempoPromedioReten'] = df['tiempoPromedioReten'].astype(float)
    df['tiempoPromedioAbandono'] = df['tiempoPromedioAbandono'].astype(float)



    #Concatenar df con df_NDS
    df_final = pd.concat([df_final, df], ignore_index=True) 

    return df_final


def leer_procesar_CALLS(fecha):

    pyautogui.write(fecha)
    time.sleep(0.05)
    pyautogui.press("tab")
    time.sleep(0.05)
    pyautogui.press("tab")
    time.sleep(0.05)
    pyautogui.press("delete")
    time.sleep(0.5)
    pyperclip.copy("CAPTA FTTH GSS;MIGRA PRE POS GSS;PORTA HOGAR GSS;PROSPECT ALTA GSS;DESBORDE CROSS A GSS;TRANSFER CROSS GSS;DESBOR TRAN CROS GSS;HOGAR COMERCIAL GSS;MOVIL COMERCIAL GSS;DESBORDE COM A GSS;TRANSF COM HOGAR GSS;TRANSF COM MOVIL GSS;DESBORD TRANS A GSS")
    time.sleep(0.5)
    pyautogui.hotkey('ctrl', 'v')
    pyautogui.press("enter")
    time.sleep(3)
    pyautogui.press("alt")
    time.sleep(0.05)
    pyautogui.press("right")
    time.sleep(0.05)
    pyautogui.press("enter")
    time.sleep(0.05)
    pyautogui.press("down")
    time.sleep(1)
    pyautogui.press("down")
    time.sleep(1)
    pyautogui.press("enter")
    time.sleep(1)
    pyautogui.press("enter")
    time.sleep(1)
    pyautogui.press("enter")
    time.sleep(5)

    
    
    #Leer reporte
    df = pd.read_clipboard()
    #Filtrar el df quedandome con todo menos la primer fila que es el total
    df = df.iloc[1:]

    # Asignar el skill correspondiente a cada vdn

    vdnCross = [39802,39804,39803,39805,37012,37013,37014]

    vdnCom = [39800,39801,37000,37001,37002,37011]

    def check_skill(vdn):
        if vdn in vdnCross:
            return 861
        elif vdn in vdnCom:
            return 860 
        else: 
            return np.nan
            
    df['SKILL'] = df['VDN'].apply(check_skill)


    #Genero la columna de ofrecidas
    df["Ofrecidas"] = df["INCALLS"] - df["ABNCALLS1"]

    # Me quedo solo con las columnas de interes

    columnas = ['ROW_DATE', 'SKILL', 'STARTTIME', 'Ofrecidas']
    df = df[columnas]

    # AGRUPAMOS POR FECHA, INTERVALO Y SKILL RESUMIEDO POR SUMA
    df = df.groupby(['ROW_DATE', 'STARTTIME', 'SKILL']).sum().reset_index()


    nuevos_nombres = {
                'ROW_DATE': 'Fecha',
                'STARTTIME':'Intervalo',
                'SKILL':'Skill'
                }

    # Usamos el método 'rename' para cambiar los nombres de las columnas
    df = df.rename(columns=nuevos_nombres)

    # Convertir la columna 'Fecha' a un objeto de fecha
    df['Fecha'] = pd.to_datetime(df['Fecha'], format='%d/%m/%Y')

    # Formatear la columna 'Fecha' al formato deseado
    df['Fecha'] = df['Fecha'].dt.strftime('%Y/%m/%d')


    return df


def calculate_erlang_c(row):
    try:
        if row['Skill'] == 860:
            asa = 120 / 60  # Valor de asa para Skill 860 en minutos
            service_level = 0.7  # Valor de service_level para Skill 860
            reductor = 0.15
        elif row['Skill'] == 861:
            asa = 60 / 60  # Valor de asa para Skill 861 en minutos
            service_level = 0.8  # Valor de service_level para Skill 861
            reductor = 0.15
        else:
            # Valor de asa y service_level por defecto si Skill no es 860 ni 861
            asa = 0.0
            service_level = 0.0
        
        erlang = ErlangC(
            transactions=row['Pronostico'],
            asa=asa,
            aht=row['TMONuevo'] / 60,  # Convierte TMO a horas
            interval=30,  # Intervalo fijo en minutos
            shrinkage=reductor
        )
        requirements = erlang.required_positions(service_level=service_level, max_occupancy=0.94)
        return requirements
    except Exception as e:
        # Si ocurre un error, puedes manejarlo de la manera que desees.
        # En este ejemplo, simplemente imprimimos el error y retornamos None.
        #print(f"Error en fila {row.name}: {str(e)}")
        return None
    


def ponderador(df,column1, columnPonderadora):
    suma_productos = (df[column1] * df[columnPonderadora]).sum()
    suma_columnPonderadora = df[columnPonderadora].sum()
    resultadoPonderado = suma_productos / suma_columnPonderadora
    return resultadoPonderado


def evaluar_condiciones(nds, desvioProno, CO_real, CO_prono, desvioAusentismo, improdReal, improdProno, TMO_real, TMO_prono, incidencia, interna):
    acciones = []

    if nds >= 100: 
        if desvioProno >= 0:
            if CO_real >= 1.05:
                acciones.append("Adelantar momento T, coaching, agenda o cursos") #Si se cumple NDS y desvio en prono es mayor a 0% y CO real es mayor a 105%
            else:
                acciones.append("No accionar") #Si se cumple NDS y desvio en prono es mayor a 0% y CO real es menor a 105%          
        else:
            if desvioProno <= -0.1:
                acciones.append("Pasar asesores a OUT")  #Si se cumple NDS y desvio en prono es  menor a 0% y menor a -10%
            else:
                acciones.append("Adelantar momento T, coaching, agenda o cursos") #Si se cumple NDS y desvio en prono esta entre -10% y 0%

    else: 
        
        if CO_real >= 1.05: 
            if desvioProno >= 0:
                acciones.append("No accionar") #Si no se cumple NDS, la capacidad es mayor o igual al 105% y el desvio en el prono es mayor o igual a 0%
            else:
                if desvioProno <= -0.1:
                    acciones.append("Pasar asesores a OUT") #Si no se cumple NDS, CO real mayor al 105% y desvío en prono es menor al -10%
                else:
                    acciones.append("Adelantar momento T, coaching, agenda o cursos")  #Si no se cumple el NDS, la CO real es mayor al 105% y el desvio en prono esta entre -10% y 0%
        else:
            if CO_real >= CO_prono:
                acciones.append("No accionar") #Si no se cumple NDS, CO real menor al 105%  y la CO real es mayor a CO prono
            else:
                if desvioProno <= 0.1:
                    if desvioAusentismo >= 0.03:
                        #Si no se cumple NDS, CO real menor al 105%, la CO real es menor a CO prono y desvio en ausentismo es mayor al 3%
                        acciones.append("Desvío en ausentismo detectado: Notificar a PPP : 1. Suspender momento T y auxiliares programados. 2. Realizar blendeo.")
                    if improdReal > improdProno + 0.01:
                        #Si no se cumple NDS, CO real menor al 105%, la CO real es menor a CO prono y la improd real es mayor que la pronosticada
                        acciones.append("Desvío en improductividad detectado: Suspender auxiliares excepto baño y refrigerios")
                    if TMO_real > TMO_prono + 10:
                        #Si no se cumple NDS, CO real menor al 105%, la CO real es menor a CO prono y desvio existe un desvio en el TMO en mas de 5 seg
                        acciones.append("Desvío en TMO detectado")
                    if incidencia == "No" and not (desvioAusentismo >= 0.03) and not (improdReal > improdProno + 0.015) and not (TMO_real > TMO_prono):
                        acciones.append("Informar al PPP")
                    if incidencia == "Si" and interna == "Si" and not (desvioAusentismo >= 0.03) and not (improdReal > improdProno + 0.015) and not (TMO_real > TMO_prono):
                        acciones.append("Notificar a sistemas")
                    if incidencia == "Si" and interna == "No" and not (desvioAusentismo >= 0.03) and not (improdReal > improdProno + 0.015) and not (TMO_real > TMO_prono):
                        acciones.append("Realizar ticket correspondiente") 
                else:
                    acciones.append("No accionar")     
        
    return acciones

### Consultas

In [9]:

import os

# Obtén el directorio de trabajo actual
directorio = os.getcwd()


#------------------- CONSULTA DE REPORTE SPLIT SKILL -------------------

# Ruta completa al ejecutable
path = os.path.join(directorio, "splitSkill.acsup")

# Ejecuta el reporte 
os.startfile(path)


click_on_image_when_visible("reporte.PNG")
pyautogui.moveTo(500,500)

time.sleep(0.5)

#Consultar los spliSkill para los 3 skill
splitSkill_NDS("860", todayCMS)
df_splitSkill = leer_procesar_concatenar_SplitSkill(860,todaySQL, df_splitSkill)

splitSkill_NDS("861", todayCMS)
df_splitSkill = leer_procesar_concatenar_SplitSkill(861,todaySQL, df_splitSkill)



pyautogui.hotkey("alt","f4")


#------------------- CONSULTA DE REPORTE NDS -------------------

# Ruta completa al ejecutable
path = os.path.join(directorio, "nds.acsup")

# Ejecuta el reporte
os.startfile(path)


click_on_image_when_visible("nds.PNG")
pyautogui.moveTo(500,500)
time.sleep(0.5)

#Navegar reporte de NDS
splitSkill_NDS("860", todayCMS)
df_NDS = leer_procesar_concatenar_NDS(860,todaySQL,df_NDS)

splitSkill_NDS("861", todayCMS)
df_NDS = leer_procesar_concatenar_NDS(861,todaySQL,df_NDS)


pyautogui.hotkey("alt","f4")

#------------------- CONSULTA DE REPORTE CALLS -------------------

# Ruta completa al ejecutable
path = os.path.join(directorio, "callsCMS.acsup")

# Ejecuta el reporte
os.startfile(path)

click_on_image_when_visible("reporte.PNG")
pyautogui.moveTo(500,500)

df_Calls = leer_procesar_CALLS(todayCMS)

pyautogui.hotkey("alt","f4")

C:\Users\giuliano.colamaio\AppData\Local\Temp\ipykernel_3792\2099042009.py:127: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_final = pd.concat([df_final, df], ignore_index=True)
C:\Users\giuliano.colamaio\AppData\Local\Temp\ipykernel_3792\2099042009.py:185: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_final = pd.concat([df_final, df], ignore_index=True)


In [10]:
consultaDimensionamiento = f"""

SELECT 
dp.Fecha,
dp.Intervalo,
dp.Skill,
dp.Volumen as Pronostico,
dp.TMO as TMO_prono,
dp.Improductividad as Improductividad_prono,
dp.Ausentismo as Ausentismo_prono,
dp.Raw_Positions as Erlang,
da.RACS as Racs_prono,
da.DIM as FTE_prono,
da.PSTAFF as Pstaf_prono
from dimensionamientoppp dp
left join dimaaron da on dp.Fecha = da.FECHA and da.INTERVALO = dp.INTERVALO and da.Skill = dp.Skill
WHERE dp.Fecha = "{todaySQL}" and dp.Intervalo between "{inicio}" and "{fin}";

"""


### Consulta a base de datos de "dimensionamientoppp" y "dimaaron"

In [11]:
df_dimensionamiento = pd.read_sql_query(consultaDimensionamiento, cnx)
# Convertir la columna 'Fecha' a un objeto de fecha
df_dimensionamiento['Fecha'] = pd.to_datetime(df_dimensionamiento['Fecha'])

# Formatear la columna 'Fecha' al formato deseado
df_dimensionamiento['Fecha'] = df_dimensionamiento['Fecha'].dt.strftime('%Y/%m/%d')

cnx.close()

C:\Users\giuliano.colamaio\AppData\Local\Temp\ipykernel_3792\3981746905.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_dimensionamiento = pd.read_sql_query(consultaDimensionamiento, cnx)


### Merge de los dataframe en uno solo

In [12]:
result1 = pd.merge(df_dimensionamiento, df_splitSkill, on=['Fecha', 'Intervalo', 'Skill'], how='left')

result2 = pd.merge(result1, df_NDS, on=['Fecha', 'Intervalo', 'Skill'], how='left')

df_final = pd.merge(result2, df_Calls, on=['Fecha', 'Intervalo', 'Skill'], how='left')

df_final

,Fecha,Intervalo,Skill,Pronostico,TMO_prono,Improductividad_prono,Ausentismo_prono,Erlang,Racs_prono,FTE_prono,...,NDS,Llegadas,Abandonadas,Flujodesalida,tiempoPromedioAbandono,velPromedioDeRespuesta,demoraMaxima,tiempoPromedioReten,entregasInterrumpidas,Ofrecidas
0,2024/08/27,17:00,860,216,455,0.158150,0.072087,58.0,62,48.43,...,99.0,169,1,0,69.0,16.801243,107,128.445953,0,165
1,2024/08/27,17:00,861,46,464,0.151986,0.143243,15.0,19,13.80,...,100.0,40,0,0,NaN,0.028571,1,73.466667,0,35


### Calculo de nececidad real

In [13]:
df_final['NecesidadReal'] = df_final.apply(calculate_erlang_c, axis=1).apply(lambda x: x['raw_positions'] if x is not None else 0)


### Procesamiento de df_final

In [14]:
#Calculo columnas 

df_final['FTE_reales'] = (df_final['TiempoDisponible'] + df_final['TiempoRing'] + df_final['TiempoACD'] + df_final['TiempoAcw'] + df_final['Tiempodereten'] + df_final['TiempoLlamadaSaliente']) / 1800

df_final["CO_pronosticada"] = 1 + (df_final["FTE_prono"] / df_final["Erlang"] - 1)

df_final['CO_real'] = df_final["FTE_reales"] / df_final["NecesidadReal"]

# Crear la columna Ausentismo
df_final['Ausentismo_real'] = 1 - df_final['PosicionStaff'] / df_final['Racs_prono']
# Reemplazar valores negativos con 0
df_final['Ausentismo_real'] = df_final['Ausentismo_real'].apply(lambda x: max(x, 0))


df_final["Improductividad_real"] = 1 - df_final["PosicionStaff"] / df_final["FTE_reales"]


# Crear la columna Desvio
df_final['Desvio'] = df_final['Ofrecidas'] / df_final['Pronostico'] - 1


columnas = ['Fecha', 'Intervalo', 'Skill', 'Pronostico', 'TMO_prono',
            'Improductividad_prono', 'Ausentismo_prono', 'Erlang', 'Racs_prono',
            'FTE_prono', 'PosicionStaff','LlamadasAtendidas', 'TMONuevo', 'TiempoAux', 
            'AgentesDisponibles','NDS', 'Llegadas', 'Ofrecidas', 'FTE_reales', 'CO_pronosticada', 
            'Ausentismo_real', 'Improductividad_real','Desvio','NecesidadReal', "CO_real", 'Pstaf_prono'
            ]


df_final = df_final[columnas]

# df_final.to_excel("test.xlsx", index=False)

df_final.head()

,Fecha,Intervalo,Skill,Pronostico,TMO_prono,Improductividad_prono,Ausentismo_prono,Erlang,Racs_prono,FTE_prono,...,Llegadas,Ofrecidas,FTE_reales,CO_pronosticada,Ausentismo_real,Improductividad_real,Desvio,NecesidadReal,CO_real,Pstaf_prono
0,2024/08/27,17:00,860,216,455,0.158150,0.072087,58.0,62,48.43,...,169,165,45.338889,0.835,0.129032,-0.191031,-0.236111,64,0.708420,57.53
1,2024/08/27,17:00,861,46,464,0.151986,0.143243,15.0,19,13.80,...,40,35,13.656111,0.920,0.000000,-0.391318,-0.239130,15,0.910407,16.28


### Publicar df_final

In [15]:
if publicar == 1:
    # Definir nombre de la tabla
    tabla_df_final = "datadecisiones"

    # Iniciar conexión
    cnx = iniciarconexion('26.89.137.57', 'db_atp', 'gtr1', 'sqlPassGTR2024*')

    #Si la columna desvio tiene un valor nulo lo remplazo por 0
    df_final["Desvio"] = df_final["Desvio"].fillna(0)
    # Armado de tupla
    data = [tuple(row) for row in df_final.values]

    # Columnas
    columns = df_final.columns.tolist()
    columns_str = ', '.join(columns)

    # Placeholders
    placeholders = ', '.join(['%s' for _ in columns])

    # SQL de inserción
    sql = f'INSERT INTO {tabla_df_final} ({columns_str}) VALUES ({placeholders})'

    # Crear cursor
    cursor = cnx.cursor()

    # Ejecutar inserciones
    cursor.executemany(sql, data)

    # Confirmar cambios
    cnx.commit()

    # Cerrar conexión
    cursor.close()
    cnx.close()




C:\Users\giuliano.colamaio\AppData\Local\Temp\ipykernel_3792\1454781735.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final["Desvio"] = df_final["Desvio"].fillna(0)


### Construyo dos dataframes uno para 860 y otro para 861

In [16]:
# Filtrar por Skill 860
df_860 = df_final.loc[df_final['Skill'] == 860]

# Filtrar por Skill 861
df_861 = df_final.loc[df_final['Skill'] == 861]


### Calculo de variables para 860

In [17]:
nds_860 = round(float(ponderador(df_860, "NDS", "Llegadas")),3)

tmo_860_real = round(ponderador(df_860, "TMONuevo", "LlamadasAtendidas"),3)
tmo_860_prono = df_860["TMO_prono"].mean().round(3)
desvioTMO_860 = (tmo_860_real / tmo_860_prono -1)

improdReal_860 = (1 - df_860["FTE_reales"].sum() / df_860["PosicionStaff"].sum()).round(3)
improdProno_860 = (1 - df_860["FTE_prono"].sum() / df_860["Pstaf_prono"].sum()).round(3)
desvioImprod_860 = (improdReal_860 / improdProno_860 - 1).round(3)


CO_prono_860 = df_860["CO_pronosticada"].mean().round(3)
CO_real_860 = df_860["CO_real"].mean().round(3)

desvioCO_860 = CO_real_860 / CO_prono_860 - 1
desvioProno_860 = (df_860["Ofrecidas"].sum()  / df_860["Pronostico"].sum() - 1).round(3)

racsProno_860 = df_860["Racs_prono"].sum()
desvioAusentismo_860 =  (df_860["Pstaf_prono"].sum() / df_860["PosicionStaff"].sum() - 1).round(3)

descripcionVariables_860 = f'NDS: {nds_860}\nCO_real: {CO_real_860}\nCO_prono: {CO_prono_860}\nDesvio_prono: {desvioProno_860}\nTMO_real: {tmo_860_real}\nTMO_prono: {tmo_860_prono}\nIprod real: {improdReal_860}\nImprod_prono: {improdProno_860}\nDesvio_Improd: {desvioImprod_860}\nDesvio_Ausentismo: {desvioAusentismo_860}'
descripcionVariables_860

'NDS: 99.0\nCO_real: 0.708\nCO_prono: 0.835\nDesvio_prono: -0.236\nTMO_real: 495.0\nTMO_prono: 455.0\nIprod real: 0.16\nImprod_prono: 0.158\nDesvio_Improd: 0.013\nDesvio_Ausentismo: 0.065'

### Calculo de variables para 861

In [18]:
nds_861 = round(float(ponderador(df_861, "NDS", "Llegadas")),3)

tmo_861_real = round(ponderador(df_861, "TMONuevo", "LlamadasAtendidas"),3)
tmo_861_prono = df_861["TMO_prono"].mean().round(3)
desvioTMO_861 = (tmo_861_real / tmo_861_prono -1)

improdReal_861 = (1 - df_861["FTE_reales"].sum() / df_861["PosicionStaff"].sum()).round(3)
improdProno_861 = (1 - df_861["FTE_prono"].sum() / df_861["Pstaf_prono"].sum()).round(3)
desvioImprod_861 = (improdReal_861 / improdProno_861 - 1).round(3)


CO_prono_861 = df_861["CO_pronosticada"].mean().round(3)
CO_real_861 = df_861["CO_real"].mean().round(3)


desvioCO_861 = CO_real_861 / CO_prono_861 - 1
desvioProno_861 = (df_861["Ofrecidas"].sum()  / df_861["Pronostico"].sum() - 1).round(3)

racsProno_861 = df_861["Racs_prono"].sum()
desvioAusentismo_861 =  (  df_861["Pstaf_prono"].sum() /df_861["PosicionStaff"].sum() -1 ).round(3)


descripcionVariables_861 =f'NDS: {nds_861}\nCO_real: {CO_real_861}\nCO_prono: {CO_prono_861}\nDesvio_prono: {desvioProno_861}\nTMO_real: {tmo_861_real}\nTMO_prono: {tmo_861_prono}\nIprod real: {improdReal_861}\nImprod_prono: {improdProno_861}\nDesvio_Improd: {desvioImprod_861}\nDesvio_Ausentismo: {desvioAusentismo_861}'
descripcionVariables_861

'NDS: 100.0\nCO_real: 0.91\nCO_prono: 0.92\nDesvio_prono: -0.239\nTMO_real: 450.0\nTMO_prono: 464.0\nIprod real: 0.281\nImprod_prono: 0.152\nDesvio_Improd: 0.849\nDesvio_Ausentismo: -0.143'

### Evaluación de condiciones

In [19]:

# Evaluamos las condiciones
# Al evaluar condicionees obtenemos una lista con cada una de las acciones para cada skill 


acciones860 = evaluar_condiciones(nds_860, desvioProno_860, CO_real_860, CO_prono_860, desvioAusentismo_860, improdReal_860, improdProno_860, tmo_860_real, tmo_860_prono, incidencia, interna)

acciones861 = evaluar_condiciones(nds_861, desvioProno_861, CO_real_861, CO_prono_861, desvioAusentismo_861, improdReal_861, improdProno_861, tmo_861_real, tmo_861_prono, incidencia, interna)

print(f'Varibales 860:\n\n{descripcionVariables_860}\n\nVariables 861:\n\n{descripcionVariables_861}\n\n')
print(f'Acciones 860:\n{acciones860}\n\nAcciones 861:\n{acciones861}')

Varibales 860:

NDS: 99.0
CO_real: 0.708
CO_prono: 0.835
Desvio_prono: -0.236
TMO_real: 495.0
TMO_prono: 455.0
Iprod real: 0.16
Improd_prono: 0.158
Desvio_Improd: 0.013
Desvio_Ausentismo: 0.065

Variables 861:

NDS: 100.0
CO_real: 0.91
CO_prono: 0.92
Desvio_prono: -0.239
TMO_real: 450.0
TMO_prono: 464.0
Iprod real: 0.281
Improd_prono: 0.152
Desvio_Improd: 0.849
Desvio_Ausentismo: -0.143


Acciones 860:
['Desvío en ausentismo detectado: Notificar a PPP : 1. Suspender momento T y auxiliares programados. 2. Realizar blendeo.', 'Desvío en TMO detectado']

Acciones 861:
['Pasar asesores a OUT']


### Crear consulta con detalle de acciones para su publicación en base de datos 

In [20]:
if publicar == 1:
    try:
        import mysql.connector
        from mysql.connector import Error

        def iniciarconexion(host, database, username, password):
            try:
                cnx = mysql.connector.connect(
                    host=host,
                    database=database,
                    user=username,
                    password=password
                )
                if cnx.is_connected():
                    print("Conexión exitosa")
                    return cnx
            except Error as e:
                print(f"Error al conectar a la base de datos: {e}")
                return None

        def insertar_datos(cursor, datos):
            try:
                query = """INSERT INTO decisionesgtr (Fecha, Inicio, Fin, Skill, Incidencia, Interna, Decision, Cantidad_acciones, Timestamp) 
                        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, NOW())"""
                cursor.execute(query, datos)
            except Error as e:
                print(f"Error al insertar datos: {e}")

        # Iniciar Conexión
        cnx = iniciarconexion('26.89.137.57', 'db_atp', 'gtr1', 'sqlPassGTR2024*')
        if cnx is not None:
            cursor = cnx.cursor()

            # Insertar datos si hay acciones para 861
            if len(acciones861) > 0:
                decision861 = " ".join(acciones861)  # Unir todas las acciones en una sola cadena
                datos861 = (todaySQL, inicio, fin, 861, incidencia, interna, decision861, len(acciones861))
                print(f"Ejecutando consulta: {datos861}")
                insertar_datos(cursor, datos861)

            # Insertar datos si hay acciones para 860
            if len(acciones860) > 0:
                decision860 = " ".join(acciones860)  # Unir todas las acciones en una sola cadena
                datos860 = (todaySQL, inicio, fin, 860, incidencia, interna, decision860, len(acciones860))
                print(f"Ejecutando consulta: {datos860}")
                insertar_datos(cursor, datos860)

            # Confirmar cambios
            cnx.commit()

            # Cerrar conexión
            cursor.close()
            cnx.close()
        else:
            print("No se pudo establecer la conexión con la base de datos.")
    except Exception as e:
        print(e)


Conexión exitosa
Ejecutando consulta: ('2024/08/27', '17:00', '17:00', 861, 'No', 'No', 'Pasar asesores a OUT', 1)
Ejecutando consulta: ('2024/08/27', '17:00', '17:00', 860, 'No', 'No', 'Desvío en ausentismo detectado: Notificar a PPP : 1. Suspender momento T y auxiliares programados. 2. Realizar blendeo. Desvío en TMO detectado', 2)


In [21]:
# Marca el final del tiempo
end_time = time.time()

# Calcula el tiempo de ejecución
tiempoDeEjecucion = end_time - start_time

print(f"El tiempo de ejecución del script fue de {tiempoDeEjecucion:.2f} segundos.")

El tiempo de ejecución del script fue de 47.60 segundos.


### INTERFACE GRAFICA CON RESULTADOS

In [22]:


# Establecer el modo oscuro
ctk.set_appearance_mode("dark")  # Opciones: "dark", "light", "system"
ctk.set_default_color_theme("dark-blue")  # Puedes cambiar el tema de color según tus preferencias

# Datos de ejemplo basados en los resultados proporcionados
variables_860 = {
    "NDS": f'{int(nds_860)} %',
    "CO real": f'{int(round(CO_real_860*100,1))} %',
    "CO prono": f'{int(round(CO_prono_860*100,1))} %',
    "Desvio prono": f'{round(desvioProno_860*100,1)} %',
    "TMO real": int(round(tmo_860_real, 0)),
    "TMO prono": int(round(tmo_860_prono, 0)),
    "Improd real": f'{round(improdReal_860*100,1)} %',
    "Improd prono": f'{round(improdProno_860*100,1)} %',
    "Racs prono" : int(racsProno_860),
    "P.Staff real": int(df_860["PosicionStaff"].sum()),
    "P.Staff prono": float(df_860["Pstaf_prono"].sum()),
    "Desvio ausentismo": f'{round(desvioAusentismo_860*100,1)} %',
    
}

acciones_860 = acciones860

variables_861 = {
    "NDS": f'{int(nds_861)} %',
    "CO real": f'{int(round(CO_real_861*100,1))} %',
    "CO prono": f'{int(round(CO_prono_861*100,1))} %',
    "Desvio prono": f'{round(desvioProno_861*100,1)} %',
    "TMO real": int(round(tmo_861_real, 0)),
    "TMO prono": int(round(tmo_861_prono, 0)),
    "Improd real": f'{round(improdReal_861*100,1)} %',
    "Improd prono": f'{round(improdProno_861*100,1)} %',
    "Racs prono" : int(racsProno_861),
    "P.Staff real": int(df_861["PosicionStaff"].sum()),
    "P.Staff prono": float(df_861["Pstaf_prono"].sum()),
    "Desvio ausentismo": f'{round(desvioAusentismo_861*100,2)} %',
}

acciones_861 = acciones861

# Función para exportar el DataFrame a un archivo Excel
def export_report():
    file_path = filedialog.asksaveasfilename(defaultextension=".xlsx", initialfile="data.xlsx", filetypes=[("Excel files", "*.xlsx"), ("All files", "*.*")])
    if file_path:
        df_final.to_excel(file_path, index=False)
        print(f"Reporte guardado en {file_path}")

        
# Crear la ventana principal
root = ctk.CTk()
root.title("Resultados del Proceso")

# Configurar la ventana en pantalla completa
root.state('zoomed')  # Esto es específico de Windows, en otros sistemas operativos puedes usar: root.attributes('-fullscreen', True)

# Crear un marco principal con bordes
main_frame = ctk.CTkFrame(root, corner_radius=10)
main_frame.pack(padx=10, pady=10, expand=True, fill="both")

# Crear un título
title_label = ctk.CTkLabel(main_frame, text="Resultados", font=("Helvetica", 18, "bold"))
title_label.pack(pady=10)

# Ajustar el tamaño de los marcos
frame_width = 200
frame_height = 600

# Crear un frame para el marco de Variables 860
frame_860 = ctk.CTkFrame(main_frame, width=frame_width, height=frame_height, corner_radius=10)
frame_860.pack(side="left", padx=10, pady=10, expand=True, fill="both")

# Crear un frame para el marco de Variables 861
frame_861 = ctk.CTkFrame(main_frame, width=frame_width, height=frame_height, corner_radius=10)
frame_861.pack(side="right", padx=10, pady=10, expand=True, fill="both")

# Añadir contenido al marco de Variables 860
ctk.CTkLabel(frame_860, text="Variables 860:", font=("Helvetica", 16, "bold")).pack(padx=10, pady=10)
for key, value in variables_860.items():
    ctk.CTkLabel(frame_860, text=f"{key}: {value}", font=("Helvetica", 14), wraplength=frame_width-20).pack(padx=10, pady=5)

ctk.CTkLabel(frame_860, text="Acciones 860:", font=("Helvetica", 16, "bold")).pack(padx=10, pady=10)
for action in acciones_860:
    ctk.CTkLabel(frame_860, text=action, font=("Helvetica", 14), wraplength=frame_width-20).pack(padx=10, pady=5)

# Añadir contenido al marco de Variables 861
ctk.CTkLabel(frame_861, text="Variables 861:", font=("Helvetica", 16, "bold")).pack(padx=10, pady=10)
for key, value in variables_861.items():
    ctk.CTkLabel(frame_861, text=f"{key}: {value}", font=("Helvetica", 14), wraplength=frame_width-20).pack(padx=10, pady=5)

ctk.CTkLabel(frame_861, text="Acciones 861:", font=("Helvetica", 16, "bold")).pack(padx=10, pady=10)
for action in acciones_861:
    ctk.CTkLabel(frame_861, text=action, font=("Helvetica", 14), wraplength=frame_width-20).pack(padx=10, pady=5)

# Crear un marco para posicionar el botón en la parte inferior izquierda
button_frame = ctk.CTkFrame(root, corner_radius=0, fg_color="transparent")
button_frame.pack(side="bottom", anchor="sw", fill="x")

# Añadir botón para descargar el reporte
download_button = ctk.CTkButton(button_frame, text="Descargar Reporte", command=export_report)
download_button.pack(side="left", padx=10, pady=10)


# Ejecutar la aplicación
root.mainloop()
